In [1]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *
from sklearn.metrics import *
from scipy.stats import linregress

from sklearn.metrics.pairwise import *

import time

# Read csvs

In [2]:
%%time
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
ings = ings.drop_duplicates(subset='ingredient', keep='last')
ings['ewg'] = ings['ewg'].str[-1:]
ings['ewg'] = ings['ewg'].convert_objects(convert_numeric=True)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])
df = dataPrep(ings, prods, prod_ing)

print('Number of products df: ', len(df))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

/Users/isabelbeard/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Number of products df:  15945
Number of unique ingredients:  6899
Number of total ingredients:  446618
CPU times: user 5.94 s, sys: 282 ms, total: 6.22 s
Wall time: 6.36 s


In [23]:
product ='SKIN PERFECTING 8% AHA Gel'

##get indices of all products
indices = pd.Series(df.index, index=df['product'])#.drop_duplicates()
##find index of specified product
idx = indices[product]

print('Product:', df.iloc[idx]['product'])
print('Brand:', df.iloc[idx]['brand'])
print('Id:', idx)
print('Number of ingredients:', df.iloc[idx]['ingCount'])
print('Ingredients', df.iloc[idx]['ingList'])

Product: SKIN PERFECTING 8% AHA Gel
Brand: Paula's Choice
Id: 1422
Number of ingredients: 15
Ingredients ['Water', 'Glycolic Acid', 'Sodium Hydroxide', 'Chamomilla Recutita (Matricaria) Flower Extract', 'Aloe Barbadensis Leaf Juice', 'Camellia Oleifera Leaf Extract', 'Sodium Hyaluronate', 'Panthenol', 'Sodium PCA', 'Propylene Glycol', 'Butylene Glycol', 'Hydroxyethylcellulose', 'Polyquaternium-10', 'Phenoxyethanol', 'Sodium Benzoate']


In [24]:
###############################################################################################RBO
t0 = time.time()
df_cos = df
column = 'product'

# Get the index of the product that matches the product name
df_cos['ing#List'] = df_cos['ing#List'].astype(str)
tfidf = TfidfVectorizer(binary = True, stop_words = None)
tfidf_matrix = tfidf.fit_transform(df_cos['ing#List'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Get the pairwsie similarity scores of all products with that product
sim_scores_all_cos = list(enumerate(cosine_sim[idx]))

# Sort the products based on the similarity scores
sim_scores_sorted_cos = sorted(sim_scores_all_cos, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten_cos = sim_scores_sorted_cos[1:21]

# Get the product indices
product_indices_cos = [i[0] for i in sim_scores_ten_cos]

# Return the top 10 most similar products and their ingredients
recommended_cos = df[['product', 'brand','ingList']].iloc[product_indices_cos]

topTen_cos = pd.DataFrame(recommended_cos)
commonItems(product, topTen_cos, df)
topTen_cos = topTen_cos.reset_index()

t1 = time.time()
total_cos = t1-t0
print('Cos time ', total_cos)

topTen_ROB = topTen_cos

sim_scores_df = pd.DataFrame(sim_scores_ten_cos, columns= ['index', 'score'])
topTen_ROB = pd.merge(topTen_ROB, sim_scores_df, on='index', how = 'inner')

topTen_ROB = pd.merge(left = topTen_ROB, right = df, on= 'product', how = 'inner')
topTen_ROB = topTen_ROB.drop(columns = ['brand_y', 'ingList_y'], axis = 1)
topTen_ROB = topTen_ROB.rename(columns = {'brand_x': 'brand', 'ingList_x': 'ingList'})

for index, row in topTen_ROB.iterrows():
    #print(topTen_ROB.iloc[index,1])
    list_of_values = topTen_ROB.loc[index, 'ingList']
    values = getScores(list_of_values, ings)
    topTen_ROB.loc[index, 'toxLowrisk'] = values[0]
    topTen_ROB.loc[index, 'toxMedRisk'] = values[1]
    topTen_ROB.loc[index, 'toxHighRisk'] = values[2]
    topTen_ROB.loc[index, 'toxNA'] = values[3]

cols = ['index', 'product', 'brand', 'price', 'rating', 'score', 'toxLowrisk', 'toxMedRisk', 'toxHighRisk', 'toxNA']
topTen_ROB_results = topTen_ROB[cols]
topTen_ROB_results

Cos time  20.699974060058594


,index,product,brand,price,rating,score,toxLowrisk,toxMedRisk,toxHighRisk,toxNA
0,2744,C-Balancing Toner for Normal to Oily Skin,Obagi,NaN,0.0,0.492556,0.600000,0.300000,0.000000,0.100000
1,4677,Glycolic 10% Toning Complex,DDF - Doctor's Dermatologic Formula,NaN,0.0,0.417536,0.687500,0.187500,0.062500,0.062500
2,6243,Glycolic Toning Complex,DDF - Doctor's Dermatologic Formula,NaN,0.0,0.417536,0.687500,0.187500,0.062500,0.062500
3,5294,"Gel Plus, 15 AHA",NeoStrata,NaN,0.0,0.414078,0.714286,0.285714,0.000000,0.000000
4,8863,Serious GlyPeel - Peel Off Mask,Neova,NaN,0.0,0.406078,0.793103,0.137931,0.034483,0.034483
5,11578,MSM Skin and Facial Gel Lotion,Kala Health,NaN,0.0,0.380441,1.000000,0.000000,0.000000,0.000000
6,7113,Toning Solution Level 1,NeoStrata,NaN,0.0,0.374453,0.625000,0.375000,0.000000,0.000000
7,4975,Blemish Spot Gel,NeoStrata,NaN,0.0,0.366045,0.666667,0.333333,0.000000,0.000000
8,6514,Green Tea Cooling Eye Gel,Nuance Salma Hayek,NaN,0.0,0.362305,0.947368,0.000000,0.000000,0.052632
9,5273,"Oily Skin Solution, 8 AHA",NeoStrata,NaN,0.0,0.356817,0.666667,0.333333,0.000000,0.000000


In [25]:
topTen_ROB_results.to_csv('results/prod5Results.csv')